In [ ]:
# Imports
from eegclassify import main, load, clean, features, preprocess

# Set this to True to run on testing data
simulate_test = False
if simulate_test:
    import os
    os.environ['PYTEST_CURRENT_TEST'] = "true"

In [ ]:
%%javascript
document.title='erb-thesis/Main - Jupyter'

In [ ]:
# Load data and save into special variable that won't be overwritten (since loading takes a while)
df_loaded = load.load_labeled_eeg2()

In [ ]:
# Split into sessions (by date) for LORO CV

df_loaded.describe()

In [ ]:
# Preprocess

df = df_loaded
df = preprocess.split_rows(df, min_duration=5)
#df = clean.clean(df)
df

In [ ]:
#X, y = main.df_to_vectors(df)
#main.pca(X, y)

In [ ]:
# df = _remove_rare(df, "class", threshold_count=50)
df = clean._select_classes(
    df,
    "class",
    ["Editing->Code", "Editing->Prose"] #, "GitHub->Issues", "GitHub->Pull request"],
)

In [ ]:
# Train

try:
    main._train_raw(df)
except Exception as e:
    # TODO: Fix testing data such that it doesn't err
    print("Error while training", e)

In [ ]:
try:
    main._train_features(df)
except Exception as e:
    # TODO: Fix testing data such that it doesn't err
    print("Error while training", e)